<a href="https://colab.research.google.com/github/lfzuloaga-dot/Proyecto-Santa-Barbara/blob/main/Santa_Barbara.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install numpy_financial

In [ ]:
import pandas as pd
from numpy_financial import irr, npv

# -------------------------------
# 1. CONFIGURACIÓN DE VARIABLES
# -------------------------------
meses = [ 'Nov-2025', 'Dic-2025', 'Ene-2026', 'Feb-2026', 'Mar-2026',
         'Abr-2026', 'May-2026', 'Jun-2026', 'Jul-2026', 'Agost-2026',
         'Sep-2026', 'Oct-2026', 'Nov-2026', 'Dic-2026', 'Ene-2027']

costos_fijos_base = 5100
costos_variables_base = 2040
ingreso_base = 15000
gasto_inversion_total = 150000
meses_inversion = 3
gasto_mensual_inversion = gasto_inversion_total / meses_inversion

inflacion = {'optimista': 0.10, 'base': 0.075, 'pesimista': 0.05}  # buscar tasas de crecimiento realistas
crecimiento_real_ingresos = {'optimista': 0.10, 'base': 0.075, 'pesimista': 0.005}
tc_inicial = 249.20
tc_crecimiento = {'optimista': 0.08, 'base': 0.12, 'pesimista': 0.15}
dolarizacion = {'optimista': 0.65, 'base': 0.70, 'pesimista': 0.75}

# Impuestos
tasa_actividad_economica = 0.03  # 3% sobre ingresos
valor_ut = 43.00  # Valor actual de la Unidad Tributaria en Bs
islr_tramos = [
    (0, 600000, 0.15, 0),
    (600001, 900000, 0.22, 42000),
    (900001, float('inf'), 0.34, 150000)
]

# -------------------------------
# 2. FUNCIÓN DE SIMULACIÓN
# -------------------------------
def simular_escenario(nombre):
    infl = inflacion[nombre]
    crec_tc = tc_crecimiento[nombre]
    dol = dolarizacion[nombre]
    crec_real = crecimiento_real_ingresos[nombre]

    resultados = []
    flujo_acum = 0
    break_even = None

    for i in range(len(meses)):
        f_infl = (1 + infl) ** i
        f_tc = (1 + crec_tc) ** i
        tc_mes = tc_inicial * f_tc

        costos = (costos_fijos_base + costos_variables_base) * f_infl
        total_costos = costos * (1 - dol) + costos * dol

        inversion = gasto_mensual_inversion if i < meses_inversion else 0
        ingreso = ingreso_base * ((1 + infl) * (1 + crec_real)) ** i
        tributos_usd = ingreso * tasa_actividad_economica

        ganancia_neta = ingreso - (total_costos + tributos_usd + inversion)
        islr = 0
        if ganancia_neta > 0:
            ganancia_bs = ganancia_neta * tc_mes
            for minimo, maximo, tasa, sustraendo in islr_tramos:
                if minimo <= ganancia_bs <= maximo:
                    islr_bs = ganancia_bs * tasa - sustraendo
                    islr = islr_bs / tc_mes
                    break

        salida_total = total_costos + tributos_usd + inversion + islr
        flujo = ingreso - salida_total
        flujo_acum += flujo

        if flujo_acum > 0 and not break_even:
            break_even = meses[i]

        resultados.append({
            'Mes': meses[i],
            'Tipo de Cambio (Bs/USD)': round(tc_mes, 2),
            'Ingresos Proyectados (USD)': round(ingreso, 2),
            'Costos Operativos (USD)': round(total_costos, 2),
            'Tributos Actividad Económica (USD)': round(tributos_usd, 2),
            'Inversión Mensual (USD)': round(inversion, 2),
            'ISLR (USD)': round(islr, 2),
            'Flujo Neto (USD)': round(flujo, 2),
            'Flujo Acumulado (USD)': round(flujo_acum, 2)
        })

    return pd.DataFrame(resultados), break_even

# -------------------------------
# 3. CÁLCULO DE INDICADORES
# -------------------------------
wacc = 0.16

def calcular_indicadores(df):
    flujos = df['Flujo Neto (USD)'].tolist()
    tir = irr(flujos) if any(f > 0 for f in flujos) else float('-inf')
    van = sum(flujos)
    vpn = npv(wacc, flujos)
    return round(tir * 100, 2), round(van, 2), round(vpn, 2)

# -------------------------------
# 4. EJECUCIÓN
# -------------------------------
if __name__ == '__main__':
    escenarios = {esc: simular_escenario(esc) for esc in ['optimista', 'base', 'pesimista']}

    for nombre, (df, be) in escenarios.items():
        print(f"\n--- Escenario {nombre.capitalize()} ---")
        print(df.to_string(index=False))
        print(f"Mes de Punto de Equilibrio: {be if be else 'No se alcanza'}")

        tir, van, vpn = calcular_indicadores(df)
        print(f"\nIndicadores financieros:")
        print(f"TIR: {'N/A' if tir == float('-inf') else str(tir) + '%'}")
        print(f"VAN: USD {van}")
        print(f"VPN (WACC {wacc*100}%): USD {vpn}")


--- Escenario Optimista ---
       Mes  Tipo de Cambio (Bs/USD)  Ingresos Proyectados (USD)  Costos Operativos (USD)  Tributos Actividad Económica (USD)  Inversión Mensual (USD)  ISLR (USD)  Flujo Neto (USD)  Flujo Acumulado (USD)
  Nov-2025                   249.20                    15000.00                  7140.00                              450.00                  50000.0        0.00         -42590.00              -42590.00
  Dic-2025                   269.14                    18150.00                  7854.00                              544.50                  50000.0        0.00         -40248.50              -82838.50
  Ene-2026                   290.67                    21961.50                  8639.40                              658.85                  50000.0        0.00         -37336.74             -120175.24
  Feb-2026                   313.92                    26573.42                  9503.34                              797.20                      0.0     5054.